#### Part 1 - Download price of Stock Codes from ASX, put it into a Dataframe, export to CSV

In [8]:
# Import Modules
import yfinance as yf
import pandas as pd
import lxml
pd.set_option('display.max_rows', 500)

# Set Start & End Date
start_date = '2020-08-21'
end_date = '2020-08-22'

In [9]:
# Import CSV of ASX stocks into a List (add .AX so the stock code conforms to YFinance format)
# CSV file here: https://www.asx.com.au/asx/research/listedCompanies.do
asx_csv = pd.read_csv('ASX.csv')
asx_csv['ASX code'] = asx_csv['ASX code'] + ".AX"
asx_list = asx_csv['ASX code'].tolist()

# Confirm total number of stock codes - ~2119
print(len(asx_list))

# Testing only (if you don't want to use all 2119 stock codes)
asx_list = ['TLS.AX','APT.AX','Z1P.AX','NZS.AX']

# Download list of ASX stocks passing asx_list, start_date, and end_date as parameters
# (yf.download outputs to a Dataframe)
df = yf.download(asx_list, start=start_date, end=end_date, interval='1d', threads=False)

2119
[*********************100%***********************]  4 of 4 completed


In [22]:
df

Adj Close                              Close                       \
               APT.AX NZS.AX    TLS.AX Z1P.AX     APT.AX NZS.AX TLS.AX Z1P.AX   
Date                                                                            
2020-08-21  78.949997  0.058  3.020098   6.59  78.949997  0.058   3.05   6.59   

             High         ...    Low          Open                       \
           APT.AX NZS.AX  ... TLS.AX Z1P.AX APT.AX NZS.AX TLS.AX Z1P.AX   
Date                      ...                                             
2020-08-21   82.0  0.062  ...   3.03   6.43   81.5  0.056   3.04   6.63   

             Volume                              
             APT.AX   NZS.AX    TLS.AX   Z1P.AX  
Date                                             
2020-08-21  2199838  8801836  26832948  5370780  

[1 rows x 24 columns]

In [23]:
# Transpose dataframe from horizontal to vertical (flat file)
dft = df.transpose()
dft.reset_index(inplace=True)
dft

Date,level_0,level_1,2020-08-21 00:00:00
0,Adj Close,APT.AX,7.895000e+01
1,Adj Close,NZS.AX,5.800000e-02
2,Adj Close,TLS.AX,3.020098e+00
3,Adj Close,Z1P.AX,6.590000e+00
4,Close,APT.AX,7.895000e+01
5,Close,NZS.AX,5.800000e-02
6,Close,TLS.AX,3.050000e+00
7,Close,Z1P.AX,6.590000e+00
8,High,APT.AX,8.200000e+01
9,High,NZS.AX,6.200000e-02


In [24]:
# Only include Close, Low, High, Volume rows (ignore Adj Close)
# The last column is the price - we'll format this soon
dft = dft[ (dft['level_0'] == 'Close') | 
           (dft['level_0'] == 'Low') |
           (dft['level_0'] == 'High') |
           (dft['level_0'] == 'Volume') ]

In [25]:
# Rename columns
dft.columns = ['Desc','Code','Value']
dft

,Desc,Code,Value
4,Close,APT.AX,7.895000e+01
5,Close,NZS.AX,5.800000e-02
6,Close,TLS.AX,3.050000e+00
7,Close,Z1P.AX,6.590000e+00
8,High,APT.AX,8.200000e+01
9,High,NZS.AX,6.200000e-02
10,High,TLS.AX,3.070000e+00
11,High,Z1P.AX,6.680000e+00
12,Low,APT.AX,7.890000e+01
13,Low,NZS.AX,5.600000e-02


In [26]:
# Pivot the data
df_piv = dft.pivot(index='Code', columns='Desc')
df_piv

Value                               
Desc        Close    High        Low      Volume
Code                                            
APT.AX  78.949997  82.000  78.900002   2199838.0
NZS.AX   0.058000   0.062   0.056000   8801836.0
TLS.AX   3.050000   3.070   3.030000  26832948.0
Z1P.AX   6.590000   6.680   6.430000   5370780.0

In [27]:
# Remove Value index (makes it easier to access Close, High, Low, Volume)
df_piv = df_piv['Value'].reset_index()
df_piv

Desc,Code,Close,High,Low,Volume
0,APT.AX,78.949997,82.000,78.900002,2199838.0
1,NZS.AX,0.058000,0.062,0.056000,8801836.0
2,TLS.AX,3.050000,3.070,3.030000,26832948.0
3,Z1P.AX,6.590000,6.680,6.430000,5370780.0


In [ ]:
# Export to CSV (optional step - it's exported at the end anyway)
df_piv.to_csv(f'{start_date}.csv',index=False)

#### Part 2 - Append Company Info, Financials, and Ratios

In [29]:
# Go through each Stock Code in asx_list and download Company Info, Financials, Ratios (i.e., symbol, averageDailyVolume10Day, fiftyTwoWeekHigh, etc)
# try / catch is used so if the yfinance API doesn't have / can't find info on that company, it won't error out and
# it will go to the next Stock Code
for c_num, each_code in enumerate(asx_list):
    try:
        print(each_code)
        if c_num == 0:
            ticker = yf.Ticker(each_code).info
            ticker_df = pd.DataFrame.from_dict(ticker, orient='index').transpose()
            ticker_df = ticker_df[['symbol','averageDailyVolume10Day','fiftyTwoWeekHigh','fiftyTwoWeekLow','52WeekChange','twoHundredDayAverage','fiftyDayAverage','sector','longBusinessSummary','sharesOutstanding','marketCap','enterpriseValue','enterpriseToRevenue','enterpriseToEbitda','profitMargins','bookValue','priceToBook','trailingEps','netIncomeToCommon']]
        else:
            ticker = yf.Ticker(each_code).info
            print(ticker)
            ticker_df2 = pd.DataFrame.from_dict(ticker, orient='index').transpose()
            ticker_df2 = ticker_df2[['symbol','averageDailyVolume10Day','fiftyTwoWeekHigh','fiftyTwoWeekLow','52WeekChange','twoHundredDayAverage','fiftyDayAverage','sector','longBusinessSummary','sharesOutstanding','marketCap','enterpriseValue','enterpriseToRevenue','enterpriseToEbitda','profitMargins','bookValue','priceToBook','trailingEps','netIncomeToCommon']]
            ticker_df = ticker_df.append(ticker_df2)
    except:
        continue

TLS.AX
APT.AX
Z1P.AX
{'zip': '2000', 'sector': 'Financial Services', 'fullTimeEmployees': 371, 'longBusinessSummary': 'Zip Co Limited provides point-of-sale credit and digital payment services to consumers and merchants in Australia and New Zealand. It offers integrated retail finance solutions to merchants in the retail, home, health, automotive, and travel industries through online and in-store. The company provides zip Pay and zip Money, which are digital wallets; and Pocketbook, a mobile app that automatically categorizes spending, as well as unsecured loans to small and medium sized businesses. The company was formerly known as ZipMoney Limited and changed its name to Zip Co Limited in December 2017. Zip Co Limited was incorporated in 2009 and is based in Sydney, Australia.', 'city': 'Sydney', 'state': 'NSW', 'country': 'Australia', 'companyOfficers': [], 'website': 'http://zipmoneylimited.com.au', 'maxAge': 1, 'address1': '10 Spring Street', 'industry': 'Credit Services', 'addres

In [32]:
# Export to CSV (optional step - it's exported at the end anyway)
ticker_df.to_csv('ticker_df.csv', index=False)

In [4]:
# Merge the 2 dataframes together - df_piv and ticker_df - and export to CSV
# df_piv = Price Info
# ticker_df = Company Info, Financials, and Ratios
# Join these two Dataframes by the 'symbol' column
pd.set_option('display.max_rows', 2000)
df_piv.merge(ticker_df, left_on='Code', right_on='symbol', how='left').to_csv('today.csv',index=False)